In [12]:
import os, sys
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.data_loader import TMDbDataLoader
from src.preprocessing import DataPreprocessor
from src.model_trainer import ModelTrainer

plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.grid"] = True

PROJECT_ROOT


'd:\\Movie_Revenue_Prediction_v2'

In [13]:
raw_path = os.path.join(PROJECT_ROOT, "data", "raw", "movies_2020_2024.csv")

# Nếu chưa có file raw thì dùng TMDbDataLoader để tạo
if not os.path.exists(raw_path):
    loader = TMDbDataLoader()
    loader.fetch_data()  # hoặc fetch_data(start_year=2020, end_year=2024)
    loader.save_data(raw_path)

df_raw = pd.read_csv(raw_path)
df_raw.head()


,id,title,budget,revenue,runtime,release_date,vote_average,vote_count,popularity,overview,genres,production_companies,original_language,status
0,38700,Bad Boys for Life,90000000,426505244,124,2020-01-15,7.127,8553,8.0244,Marcus and Mike are forced to confront new thr...,Thriller|Action|Crime,Columbia Pictures|2.0 Entertainment|Don Simpso...,en,Released
1,577922,Tenet,205000000,365304105,150,2020-08-22,7.200,10634,6.2013,Armed with only one word - Tenet - and fightin...,Action|Thriller|Science Fiction,Warner Bros. Pictures|Syncopy,en,Released
2,454626,Sonic the Hedgehog,85000000,319715683,99,2020-02-12,7.300,10152,8.9367,"Powered with incredible speed, Sonic The Hedge...",Action|Science Fiction|Comedy|Family,Original Film|Blur Studio|Marza Animation Plan...,en,Released
3,448119,Dolittle,175000000,251410631,101,2020-01-02,6.590,3770,7.8160,"After losing his wife seven years earlier, the...",Family|Comedy|Fantasy|Adventure,Universal Pictures|Team Downey|Perfect World P...,en,Released
4,529203,The Croods: A New Age,65000000,215905815,95,2020-11-25,7.500,4016,6.9227,"Searching for a safer habitat, the prehistoric...",Animation|Family|Adventure|Fantasy|Comedy,DreamWorks Animation,en,Released


In [4]:
from src.preprocessing import DataPreprocessor

config_path = os.path.join(PROJECT_ROOT, "configs", "config.yaml")
preprocessor = DataPreprocessor(config_path=config_path)
preprocessor

# Fit trên toàn bộ dữ liệu rồi transform
X, y = preprocessor.fit_transform(df_raw)

X.shape, y.shape


2025-12-06 21:48:07,114 - src.preprocessing - INFO - DataPreprocessor đã được khởi tạo thành công (target_col=revenue, scaler=StandardScaler)
2025-12-06 21:48:07,115 - src.preprocessing - INFO - Bắt đầu fit preprocessing pipeline...


d:\Movie_Revenue_Prediction_v2\src\preprocessing.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('', inplace=True)
d:\Movie_Revenue_Prediction_v2\src\preprocessing.py:157: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

((433, 134), (433,))

In [5]:
processed_dir = os.path.join(PROJECT_ROOT, "data", "processed")
os.makedirs(processed_dir, exist_ok=True)

X_train_path = os.path.join(processed_dir, "X_train.csv")
y_train_path = os.path.join(processed_dir, "y_train.csv")

pd.DataFrame(X, columns=preprocessor.get_feature_names()).to_csv(X_train_path, index=False)
pd.DataFrame({"revenue": y}).to_csv(y_train_path, index=False)

preproc_path = os.path.join(PROJECT_ROOT, "models", "preprocessor.pkl")
os.makedirs(os.path.dirname(preproc_path), exist_ok=True)
preprocessor.save_preprocessor(preproc_path)

X_train_path, y_train_path, preproc_path


2025-12-06 21:52:36,276 - src.preprocessing - INFO - Đã lưu preprocessor vào: d:\Movie_Revenue_Prediction_v2\models\preprocessor.pkl


('d:\\Movie_Revenue_Prediction_v2\\data\\processed\\X_train.csv',
 'd:\\Movie_Revenue_Prediction_v2\\data\\processed\\y_train.csv',
 'd:\\Movie_Revenue_Prediction_v2\\models\\preprocessor.pkl')

In [9]:
import os
from pathlib import Path
from src.model_trainer import ModelTrainer

PROJECT_ROOT = Path(r"d:\Movie_Revenue_Prediction_v2")  # sửa nếu khác
config_path = PROJECT_ROOT / "configs" / "config.yaml"



trainer = ModelTrainer(config_path=str(config_path))
trainer.load_data(X_train, X_test, y_train, y_test)


2025-12-06 21:56:03,472 - src.model_trainer - INFO - ModelTrainer đã được khởi tạo thành công
2025-12-06 21:56:03,472 - src.model_trainer - INFO - Đã load data - Train: (346, 134), Test: (87, 134)


In [10]:
comparison_df = trainer.compare_models()
comparison_df


RuntimeError: Chưa có model nào được train